## fasttext

In [16]:
!pip install fasttext

In [17]:
!chmod +x your_script.sh


chmod: cannot access 'your_script.sh': No such file or directory


In [18]:
!wget https://github.com/facebookresearch/fastText/archive/master.zip
!unzip master.zip
!mv fastText-master fastText
!rm master.zip
!cd fastText && make
!mv fastText/fasttext ./fasttext


--2023-12-03 21:20:53--  https://github.com/facebookresearch/fastText/archive/master.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/refs/heads/master [following]
--2023-12-03 21:20:53--  https://codeload.github.com/facebookresearch/fastText/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.121.10
Connecting to codeload.github.com (codeload.github.com)|140.82.121.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   4.17M  12.2MB/s    in 0.3s    

2023-12-03 21:20:54 (12.2 MB/s) - ‘master.zip’ saved [4370899]

Archive:  master.zip
a20c0d27cd0ee88a25ea0433b7f03038cd728459
   creating: fastText-master/
   creating: fastText-master/.circlec

In [19]:
!./fasttext skipgram -input data/your_text_file.txt -output data/fasttext/your_model_name -dim 300 -minCount 79


terminate called after throwing an instance of 'std::invalid_argument'
  what():  data/fasttext/your_model_name.bin cannot be opened for saving.


In [20]:
!pip install gensim

!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1V4rTx4yaAg0x1NY1MpNRY2Dp1nKeyOQ7" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1V4rTx4yaAg0x1NY1MpNRY2Dp1nKeyOQ7" -o wiki_20190620_small.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
100 75.1M  100 75.1M    0     0  19.3M      0  0:00:03  0:00:03 --:--:-- 63.1M


In [21]:
#0.사용자 지정함수 설정,라이브러리 임포트
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import mean_squared_error
!pip install transformers
!pip install kiwipiepy

from kiwipiepy import Kiwi
from transformers import BertModel

def print_obj(obj, name):
    print("%s:\n%s\n" % (name, obj))

In [22]:
#1.데이터 불러오기(데이터형확인)(dim,shape)
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# Google Drive에 있는 파일 읽기
file_path = '/content/drive/MyDrive/Colab Notebooks/db.sheet.csv'
dataset = pd.read_csv(file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
print(dataset.columns)

Index(['경력', '[채용] 본교 인재개발원 연구원 채용 공고 ', '인재개발', '연구원', '채용', 'Unnamed: 5'], dtype='object')


In [24]:
from gensim.models.fasttext import FastText
from gensim.models.word2vec import Text8Corpus

file_path = '/content/drive/MyDrive/Colab Notebooks/kiwi_auto.csv'
dataset = pd.read_csv(file_path)

print(dataset.dtypes)# 각 열의 데이터 타입 확인
print(dataset.shape)# 데이터의 행과 열의 개수 확인

from gensim.models import FastText

sentences = [str(sentence).split() for column in dataset.columns for sentence in dataset[column]]

model = FastText(vector_size=50, window=5, min_count=10)
model.build_vocab(sentences)  # 단어 사전 구축
model.train(sentences, total_examples=model.corpus_count, epochs=10)  # 모델 훈련
print(model)

0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
dtype: object
(219, 21)
FastText<vocab=34, vector_size=50, alpha=0.025>


In [25]:
from gensim.models import FastText

# 모델을 저장하고 로드하는 코드
model.save('fasttext_model')
saved_model = FastText.load('fasttext_model')


# '봉사' 단어의 벡터를 가져오기
word_vector = saved_model.wv['봉사']
print(word_vector)


[ 0.00048082 -0.00638385  0.00457487  0.0005689  -0.00513379 -0.00701016
  0.00260349  0.00312916 -0.00066542  0.00254631 -0.00603792  0.01170988
  0.01046745  0.01569405  0.00331631  0.00366644  0.01283677 -0.00201974
 -0.00456438  0.00137723 -0.00716489  0.01362949  0.00455844 -0.00132929
 -0.00381354  0.00211804  0.01032748  0.00485032  0.00299967 -0.00069535
 -0.00031681 -0.00228365  0.00051672  0.00459621 -0.00353807  0.00014739
  0.00809128  0.00248764  0.00369665 -0.00469033  0.00010796  0.00116633
  0.00402463 -0.00282838 -0.00056304 -0.00311809  0.00623818  0.00243821
 -0.0056711   0.00224105]


In [26]:
def find_similar_words(word, topn=5):
    similar_words = saved_model.wv.most_similar(word, topn=topn)
    return similar_words

def compute_similarity(word1, word2):
    similarity = saved_model.wv.similarity(word1, word2)
    return similarity

# 사용자 입력 받기
word = input("Enter a word: ")
topn = int(input("Enter the number of similar words you want to find: "))

# 유사한 단어 찾기
similar_words = find_similar_words(word, topn)
print(f"Words similar to '{word}': {similar_words}")

# 두 단어의 유사도 계산
word1 = input("Enter the first word: ")
word2 = input("Enter the second word: ")

similarity = compute_similarity(word1, word2)
print(f"Similarity between '{word1}' and '{word2}': {similarity}")


Enter a word: 공대
Enter the number of similar words you want to find: 7
Words similar to '공대': [('트랙', 0.2523621618747711), ('안내', 0.22985097765922546), ('모집', 0.165998175740242), ('인재', 0.15281307697296143), ('공고', 0.14646628499031067), ('학기', 0.13283026218414307), ('장학', 0.12928983569145203)]
Enter the first word: 조교
Enter the second word: 모집
Similarity between '조교' and '모집': 0.02015415020287037


In [27]:
similar_words = saved_model.wv.most_similar("사용자관심태그 변수", topn=5)  #사용자관심태그 변수로
print(similar_words)

[('학생', 0.30468273162841797), ('행정실', 0.25458255410194397), ('지원', 0.24628077447414398), ('수업', 0.23274846374988556), ('TELOS', 0.22515280544757843)]


In [28]:
#유사도 계산
similarity1 = saved_model.wv.similarity("채용", "조교")
similarity2 = saved_model.wv.similarity("채용", "상담회")

print(f"Similarity between '채용' and '조교': {similarity1}")
print(f"Similarity between '채용' and '상담회': {similarity2}")

Similarity between '채용' and '조교': 0.15966124832630157
Similarity between '채용' and '상담회': -0.10460583865642548


In [29]:
similar_words = saved_model.wv.most_similar(positive=['교육', '언어'], negative=['채용'])
print(similar_words)


[('인재', 0.25168344378471375), ('TELOS', 0.22663600742816925), ('nan', 0.17637142539024353), ('겨울', 0.16290052235126495), ('프로그램', 0.14624616503715515), ('대학원', 0.1414770781993866), ('학부', 0.11854300647974014), ('행정실', 0.10385886579751968), ('대학', 0.08995918929576874), ('전공', 0.08305896073579788)]


In [30]:
similar_words = saved_model.wv.similar_by_word('공대')
print(similar_words)


[('트랙', 0.2523621618747711), ('안내', 0.22985097765922546), ('모집', 0.165998175740242), ('인재', 0.15281307697296143), ('공고', 0.14646628499031067), ('학기', 0.13283026218414307), ('장학', 0.12928983569145203), ('건강', 0.08693111687898636), ('겨울', 0.08144653588533401), ('행정실', 0.07960878312587738)]
